In [1]:
#!/usr/bin/env python3
"""
File: LTDT_TH06_DuongDiNganNhat.py
Description: Giải bài tập Đường đi ngắn nhất - Bài 1: Circle, Bài 2: MinTable
Author: Assistant
Date: 2024
"""

import math
import heapq
import sys

print("=== BÀI TẬP ĐƯỜNG ĐI NGẮN NHẤT - TH06 ===\n")

# ===== BÀI 1: ĐƯỜNG ĐI GIỮA CÁC HÌNH TRÒN =====
def circle_distance(c1, c2):
    """Tính khoảng cách gần nhất giữa 2 hình tròn"""
    x1, y1, r1 = c1
    x2, y2, r2 = c2
    d = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return max(0, d - r1 - r2)

def bai1_circle():
    print("=== BÀI 1: ĐƯỜNG ĐI GIỮA CÁC HÌNH TRÒN ===")
    print("📝 Tìm đường đi tốt nhất giữa các hình tròn")
    print("   - ≤50cm: Bước qua (không tính nhảy)")
    print("   - 50-60cm: Nhảy qua (tính 1 lần nhảy)")
    print("   - >60cm: Không qua được")
    print("   - Ưu tiên: ít nhảy nhất → ít hình tròn nhất\n")

    # Input từ đề bài
    print("🎯 INPUT:")
    input_data = """4 1 4
0 0 5
0 20 5
0 30 5
0 100 10"""
    print(input_data)
    print()

    lines = input_data.strip().split('\n')
    n, s, t = map(int, lines[0].split())
    s -= 1
    t -= 1

    circles = []
    for i in range(1, n + 1):
        x, y, r = map(float, lines[i].split())
        circles.append((x, y, r))

    print(f"🔢 THÔNG TIN BÀI TOÁN:")
    print(f"   Số hình tròn: {n}")
    print(f"   Đi từ hình tròn: {s+1} → Hình tròn: {t+1}")

    print(f"\n📊 THÔNG TIN CÁC HÌNH TRÒN:")
    for i, (x, y, r) in enumerate(circles):
        print(f"   Hình tròn {i+1}: tâm({x},{y}), bán kính {r}")

    # Xây dựng đồ thị
    graph = [[] for _ in range(n)]

    print(f"\n📏 TÍNH TOÁN KHOẢNG CÁCH:")
    for i in range(n):
        for j in range(i + 1, n):
            dist = circle_distance(circles[i], circles[j])
            if dist <= 60:
                status = "🟢 BƯỚC" if dist <= 50 else "🟡 NHẢY"
                print(f"   H{i+1} → H{j+1}: {dist:.1f}cm → {status}")

                if dist <= 50:
                    weight = (0, 1)  # (số_nhảy, số_hình_tròn)
                else:
                    weight = (1, 1)

                graph[i].append((j, weight))
                graph[j].append((i, weight))
            else:
                print(f"   H{i+1} → H{j+1}: {dist:.1f}cm → 🔴 KHÔNG THỂ")

    # Dijkstra với trọng số kép
    INF = (10**9, 10**9)
    dist = [INF] * n
    parent = [-1] * n
    dist[s] = (0, 0)

    pq = []
    heapq.heappush(pq, ((0, 0), s))

    print(f"\n🔍 QUÁ TRÌNH TÌM ĐƯỜNG ĐI:")
    steps = 0
    while pq:
        (jumps, circles_count), u = heapq.heappop(pq)

        if (jumps, circles_count) > dist[u]:
            continue

        steps += 1
        status = "🎯 ĐÍCH" if u == t else f"Bước {steps}"
        print(f"   {status}: Xét H{u+1} (nhảy: {jumps}, hình: {circles_count})")

        if u == t:
            break

        for v, (jump_cost, circle_cost) in graph[u]:
            new_jumps = jumps + jump_cost
            new_circles = circles_count + circle_cost

            if (new_jumps, new_circles) < dist[v]:
                old_jumps, old_circles = dist[v]
                dist[v] = (new_jumps, new_circles)
                parent[v] = u
                heapq.heappush(pq, ((new_jumps, new_circles), v))
                action = "BƯỚC" if jump_cost == 0 else "NHẢY"
                print(f"      → Cập nhật H{v+1}: {action} (nhảy: {new_jumps}, hình: {new_circles})")

    # Kết quả
    print(f"\n🎊 KẾT QUẢ:")
    if dist[t] == INF:
        print("❌ 0 - KHÔNG CÓ ĐƯỜNG ĐI")
        print("   Không thể đi từ hình tròn đầu đến hình tròn cuối")
    else:
        total_jumps, total_circles = dist[t]
        print("✅ 1 - CÓ ĐƯỜNG ĐI")
        print(f"   Tổng số lần nhảy: {total_jumps}")
        print(f"   Tổng số hình tròn đi qua: {total_circles + 1}")

        # Truy vết đường đi
        path = []
        current = t
        while current != -1:
            path.append(current)
            current = parent[current]
        path.reverse()

        print(f"\n🗺️  ĐƯỜNG ĐI CHI TIẾT:")
        total_distance = 0
        for i in range(len(path)):
            node = path[i] + 1
            if i == 0:
                print(f"   🏁 Hình tròn {node} [BẮT ĐẦU]")
            else:
                u, v = path[i-1], path[i]
                dist_uv = circle_distance(circles[u], circles[v])
                total_distance += dist_uv
                step_type = "🟡 NHẢY" if dist_uv > 50 else "🟢 BƯỚC"
                print(f"   {step_type} → Hình tròn {node} (khoảng cách: {dist_uv:.1f}cm)")

        print(f"\n📈 TỔNG KẾT:")
        print(f"   • Tổng khoảng cách: {total_distance:.1f}cm")
        print(f"   • Số lần nhảy: {total_jumps}")
        print(f"   • Số hình tròn: {total_circles + 1}")

# ===== BÀI 2: ĐƯỜNG ĐI TRÊN BẢNG =====
def bai2_mintable():
    print("\n" + "="*60)
    print("=== BÀI 2: ĐƯỜNG ĐI TRÊN BẢNG ===")
    print("📝 Tìm đường đi ngắn nhất trên bảng số")
    print("   - Chi phí = tổng giá trị các ô trung gian")
    print("   - KHÔNG tính ô đầu và ô cuối")
    print("   - Chỉ đi qua ô có giá trị ≠ 0")
    print("   - Di chuyển 4 hướng (trên, dưới, trái, phải)\n")

    # Input từ đề bài
    print("🎯 INPUT:")
    input_data = """3 4 1 1 3 4
1 2 3 4
0 3 0 2
1 2 3 4"""
    print(input_data)
    print()

    lines = input_data.strip().split('\n')
    n, m, xi, yi, xj, yj = map(int, lines[0].split())
    xi -= 1; yi -= 1; xj -= 1; yj -= 1

    grid = []
    for i in range(1, n + 1):
        row = list(map(int, lines[i].split()))
        grid.append(row)

    print(f"🔢 THÔNG TIN BÀI TOÁN:")
    print(f"   Kích thước bảng: {n} × {m}")
    print(f"   Đi từ: ({xi+1},{yi+1}) → ({xj+1},{yj+1})")

    print(f"\n📊 BẢNG SỐ:")
    print("   " + "═" * (m * 4 + 1))
    for i in range(n):
        row_str = "   ║"
        for j in range(m):
            cell = grid[i][j]
            marker = "🚫" if cell == 0 else "  "
            if i == xi and j == yi:
                marker = "🚩"
            elif i == xj and j == yj:
                marker = "🎯"
            row_str += f" {cell:2d}{marker}║"
        print(row_str)
        if i < n - 1:
            print("   " + "═" * (m * 4 + 1))
    print("   " + "═" * (m * 4 + 1))
    print("   🚩: Điểm bắt đầu, 🎯: Điểm kết thúc, 🚫: Không thể đi")

    # Kiểm tra ô bắt đầu và kết thúc
    if grid[xi][yi] == 0 or grid[xj][yj] == 0:
        print(f"\n❌ KẾT QUẢ: 0")
        print("   Ô bắt đầu hoặc kết thúc có giá trị 0")
        return

    # 4 hướng di chuyển
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    # Dijkstra
    INF = 10**9
    dist = [[INF] * m for _ in range(n)]
    parent = [[None] * m for _ in range(n)]

    dist[xi][yi] = 0
    pq = []
    heapq.heappush(pq, (0, xi, yi))

    print(f"\n🔍 QUÁ TRÌNH TÌM ĐƯỜNG ĐI:")
    steps = 0
    while pq:
        cost, x, y = heapq.heappop(pq)

        if cost > dist[x][y]:
            continue

        steps += 1
        status = "🎯 ĐÍCH" if (x == xj and y == yj) else f"Bước {steps}"
        print(f"   {status}: Xét ô ({x+1},{y+1}), chi phí hiện tại: {cost}")

        if x == xj and y == yj:
            break

        for dx, dy in directions:
            nx, ny = x + dx, y + dy

            if 0 <= nx < n and 0 <= ny < m and grid[nx][ny] != 0:
                new_cost = cost
                if not (nx == xi and ny == yi):  # Không tính ô start
                    new_cost += grid[nx][ny]

                if new_cost < dist[nx][ny]:
                    old_cost = dist[nx][ny]
                    dist[nx][ny] = new_cost
                    parent[nx][ny] = (x, y)

                    if not (nx == xj and ny == yj):  # Không push ô end
                        heapq.heappush(pq, (new_cost, nx, ny))

                    direction_symbol = ""
                    if dx == 0 and dy == 1: direction_symbol = "→"
                    elif dx == 1 and dy == 0: direction_symbol = "↓"
                    elif dx == 0 and dy == -1: direction_symbol = "←"
                    elif dx == -1 and dy == 0: direction_symbol = "↑"

                    cost_change = f"+{grid[nx][ny]}" if new_cost != cost else ""
                    print(f"      {direction_symbol} Ô ({nx+1},{ny+1}): {old_cost} → {new_cost} {cost_change}")

    # Kết quả
    print(f"\n🎊 KẾT QUẢ:")
    if dist[xj][yj] == INF:
        print("❌ 0 - KHÔNG CÓ ĐƯỜNG ĐI")
        print("   Không thể đến được điểm đích từ điểm xuất phát")
    else:
        print("✅ 1 - CÓ ĐƯỜNG ĐI")
        print(f"   Chi phí đường đi: {dist[xj][yj]}")

        # Truy vết đường đi
        path = []
        x, y = xj, yj
        while (x, y) != (xi, yi):
            path.append((x + 1, y + 1))
            x, y = parent[x][y]
        path.append((xi + 1, yi + 1))
        path.reverse()

        print(f"\n🗺️  ĐƯỜNG ĐI CHI TIẾT:")
        total_cost = 0
        for i, (px, py) in enumerate(path):
            cost_here = grid[px-1][py-1] if not ((px-1 == xi and py-1 == yi) or (px-1 == xj and py-1 == yj)) else 0

            if i == 0:
                print(f"   🏁 Ô ({px},{py}) [START] - giá trị: {grid[px-1][py-1]}")
            elif i == len(path) - 1:
                print(f"   🎯 Ô ({px},{py}) [END] - giá trị: {grid[px-1][py-1]}")
            else:
                total_cost += cost_here
                print(f"   ➡️  Ô ({px},{py}) - giá trị: {grid[px-1][py-1]} (cộng: {cost_here})")

        print(f"\n📈 TỔNG KẾT:")
        print(f"   • Tổng chi phí: {dist[xj][yj]}")
        print(f"   • Số ô đi qua: {len(path)}")
        print(f"   • Chi phí trung gian: {total_cost}")

# ===== CHẠY TẤT CẢ BÀI TẬP =====
def main():
    print("🎯 BÀI TẬP ĐƯỜNG ĐI NGẮN NHẤT - TH06")
    print("📚 Môn: Lý Thuyết Đồ Thị")
    print("👨‍💻 Phiên bản: Google Colab Online\n")

    bai1_circle()
    bai2_mintable()

    print("\n" + "="*70)
    print("🎉 HOÀN THÀNH TẤT CẢ BÀI TẬP!")
    print("✅ Bài 1: Đường đi giữa các hình tròn (CIRCLE)")
    print("✅ Bài 2: Đường đi trên bảng (MINTABLE)")
    print("\n📖 KẾT LUẬN:")
    print("   • Bài 1: Sử dụng Dijkstra với trọng số kép (số nhảy, số hình tròn)")
    print("   • Bài 2: Sử dụng Dijkstra tìm đường đi ngắn nhất trên grid")
    print("   • Cả hai đều giải quyết bài toán tối ưu hóa đường đi")

if __name__ == "__main__":
    main()

=== BÀI TẬP ĐƯỜNG ĐI NGẮN NHẤT - TH06 ===

🎯 BÀI TẬP ĐƯỜNG ĐI NGẮN NHẤT - TH06
📚 Môn: Lý Thuyết Đồ Thị
👨‍💻 Phiên bản: Google Colab Online

=== BÀI 1: ĐƯỜNG ĐI GIỮA CÁC HÌNH TRÒN ===
📝 Tìm đường đi tốt nhất giữa các hình tròn
   - ≤50cm: Bước qua (không tính nhảy)
   - 50-60cm: Nhảy qua (tính 1 lần nhảy)
   - >60cm: Không qua được
   - Ưu tiên: ít nhảy nhất → ít hình tròn nhất

🎯 INPUT:
4 1 4
0 0 5
0 20 5
0 30 5
0 100 10

🔢 THÔNG TIN BÀI TOÁN:
   Số hình tròn: 4
   Đi từ hình tròn: 1 → Hình tròn: 4

📊 THÔNG TIN CÁC HÌNH TRÒN:
   Hình tròn 1: tâm(0.0,0.0), bán kính 5.0
   Hình tròn 2: tâm(0.0,20.0), bán kính 5.0
   Hình tròn 3: tâm(0.0,30.0), bán kính 5.0
   Hình tròn 4: tâm(0.0,100.0), bán kính 10.0

📏 TÍNH TOÁN KHOẢNG CÁCH:
   H1 → H2: 10.0cm → 🟢 BƯỚC
   H1 → H3: 20.0cm → 🟢 BƯỚC
   H1 → H4: 85.0cm → 🔴 KHÔNG THỂ
   H2 → H3: 0.0cm → 🟢 BƯỚC
   H2 → H4: 65.0cm → 🔴 KHÔNG THỂ
   H3 → H4: 55.0cm → 🟡 NHẢY

🔍 QUÁ TRÌNH TÌM ĐƯỜNG ĐI:
   Bước 1: Xét H1 (nhảy: 0, hình: 0)
      → Cập nhật H2: BƯỚC